In [1]:
# Loading packages

import pandas as pd
import numpy as np
import os
import sys
import re

In [2]:
# Working directory is set to same location as where the script is saved

os.chdir(sys.path[0])

In [3]:
# Defining a function for performing data quality analysis

# Please follow the instructions for inputs:
    # file_type can equal either 'Excel' or 'Csv'
    # path is the location of the data
    # which_columns is the index for columns to examine
        # Eg. For examining columns 1-3, enter [1,2,3] (remember to include brackets, even for only one column)
    # search_criteria is the pattern to match; multiple patterns can be matched
        # Eg. Search criteria of ["004", "ABC"] (remember to include brackets, even for only one pattern)
    # action allows three options: "remove", "filter", "replace" (remember to include quotes)
        # Eg. To remove all rows which match the pattern(s), select "remove"
    # criteria_type defaults to 'regex' for regular expressions; for literal matches, change to 'literal'
    # replace_with defaults to 'None' but should be changed if "replace" action is selected
        # Eg. For search criteria of ["004", "ABC"], replace with ["ITSQ", "1_"]

# Note:
# For regex matches, include '^' at the beginning of the pattern to look at the beginning, and '$' at the end of the pattern to look at the end
# If these are not included, any part of the string will be matched to the pattern
# For literal matches, it is not possible to specify whether to look at the beginning or end of the string; only possible with regex matches

# The function will return the following outputs:
    # df: A dataframe object of the original data with additional columns for indicating where matches occurred
    # result: A dataframe object of the original data after the specified action ("remove", "filter", "replace") has been performed

def DQA(file_type, path, which_columns, search_criteria, action, criteria_type = 'regex', replace_with = None):
    
    global df
    if file_type == 'Excel':
        df = pd.read_excel(path)
    if file_type == 'Csv':
        df = pd.read_csv(path)
    
    df = df.astype(str)
    
    col_names = df.columns.values
    
    if criteria_type == 'literal':
        search_criteria = [re.escape(sub) for sub in search_criteria]
    
    append_str = '_flag'
    
    index = np.array(which_columns)-1
    index = index.tolist()
    
    col_names_search = col_names[index]
    col_names_append = [sub + append_str for sub in col_names_search]
    
    for i in range(len(col_names_append)):
        df[col_names_append[i]] = df.iloc[: , i].str.contains('|'.join(search_criteria))
    
    df = df.replace({True: 'Anomaly', False: ''})
    
    global result
    if action == 'filter':
        result = df[df.apply(lambda r: r.str.contains('Anomaly').any(), axis=1)]
        result.reset_index(inplace=True, drop=True)
        result = result.drop(df.filter(regex='_flag').columns, axis=1)
    if action == 'remove':
        result = df[df.apply(lambda r: ~r.str.contains('Anomaly').any(), axis=1)]
        result.reset_index(inplace=True, drop=True)
        result = result.drop(df.filter(regex='_flag').columns, axis=1)
    if action == 'replace':
        result = df.copy()
        for i in range(len(col_names_search)):
            for s in range(len(search_criteria)):
                result[col_names_search[i]] = result.iloc[: , i].str.replace(search_criteria[s], replace_with[s])
        result.reset_index(inplace=True, drop=True)
        result = result.drop(df.filter(regex='_flag').columns, axis=1)

In [4]:
# Testing the function
 
DQA('Excel', 'sample_data.xlsx', [1, 2, 3], ['^000' , '^454'], 'replace', replace_with = ['', ''])

# file_type = 'Excel' because data is saved as Excel file

# path = 'sample_data.xlsx' because the code knows to look in the same directory; if data is saved in a different directory/folder, then user would have to write out the entire file path

# which_columns = [1, 2, 3] in order to look for matches in columns 1-3 of the data

# search_criteria = ['^000', '^454'] in order to look for those particular patterns

# action = 'replace' in order to replace matched patterns

# replace_with = ['', ''] where pattern '^000' and '^454' will be replaced with ''

In [5]:
# Understand where a match or "Anomaly" occurred

df

,Account_ID,Event_ID,Transaction_Number,Column_A,Column_B,Column_C,Account_ID_flag,Event_ID_flag,Transaction_Number_flag
0,A0002,12000060,2460000042,004397A887,009655B320,002222C470,,,
1,A0018,12000061,2460000042,004397A888,009655B321,002222C471,,,
2,A00214,12000062,2460000042,004397A889,009655B322,002222C472,,,
3,A00320,12000063,2460000042,004397A890,009655B323,002222C473,,,
4,A00426,12000064,2460000042,004397A891,009655B324,002222C474,,,
5,000A6542,12000065,2460000042,004397A892,009655B325,002222C475,Anomaly,,
6,000A6547,12000066,2460000042,004397A893,009655B326,002222C476,Anomaly,,
7,000A6552,12000067,2460000042,004397A894,009655B327,002222C477,Anomaly,,
8,000A6557,12000068,2460000042,004397A895,009655B328,002222C478,Anomaly,,
9,000A6562,12000069,2460000042,004397A896,009655B329,002222C479,Anomaly,,


In [6]:
# See what data looks like after performing the action

result

,Account_ID,Event_ID,Transaction_Number,Column_A,Column_B,Column_C
0,A0002,12000060,2460000042,004397A887,009655B320,002222C470
1,A0018,12000061,2460000042,004397A888,009655B321,002222C471
2,A00214,12000062,2460000042,004397A889,009655B322,002222C472
3,A00320,12000063,2460000042,004397A890,009655B323,002222C473
4,A00426,12000064,2460000042,004397A891,009655B324,002222C474
5,A6542,12000065,2460000042,004397A892,009655B325,002222C475
6,A6547,12000066,2460000042,004397A893,009655B326,002222C476
7,A6552,12000067,2460000042,004397A894,009655B327,002222C477
8,A6557,12000068,2460000042,004397A895,009655B328,002222C478
9,A6562,12000069,2460000042,004397A896,009655B329,002222C479
